<a href="https://colab.research.google.com/github/A-H-Sumon/FusionNet/blob/main/MCnemarTest_NumtaDb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tensorflow


In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from joblib import Parallel, delayed
import multiprocessing

import os
import cv2
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.neighbors import NearestNeighbors
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from skimage.feature import hog
from skimage.filters import gabor
from joblib import Parallel, delayed
import multiprocessing
from termcolor import colored

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam

In [3]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"anowerhossensumon","key":"157d4cd7ff6a9132eebc558a7d7fb011"}'}

In [4]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [5]:
!kaggle datasets download -d BengaliAI/numta --unzip


Dataset URL: https://www.kaggle.com/datasets/BengaliAI/numta
License(s): CC-BY-SA-4.0
 99% 1.89G/1.91G [00:20<00:00, 65.1MB/s]
100% 1.91G/1.91G [00:20<00:00, 98.6MB/s]


In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
# ======================== Data Loading ========================
base_path = "/content"



# Load and combine all labeled training data
all_image_paths = []
all_labels = []

for file_name in os.listdir(base_path):
    if file_name.startswith("training-") and file_name.endswith(".csv"):
        train_file_path = os.path.join(base_path, file_name)
        labels_df = pd.read_csv(train_file_path)
        folder_name = file_name.replace(".csv", "")
        folder_path = os.path.join(base_path, folder_name)

        if os.path.exists(folder_path):
            for _, row in labels_df.iterrows():
                image_file = row['filename']
                label = row['digit']
                image_path = os.path.join(folder_path, image_file)
                all_image_paths.append(image_path)
                all_labels.append(label)

# ===================== Train/Test Split =====================
X_train, X_test, y_train, y_test = train_test_split(
    all_image_paths, all_labels, test_size=0.2, random_state=42, stratify=all_labels
)

# ===================== Create DataFrames =====================
train_df = pd.DataFrame({"image_path": X_train, "label": y_train})
test_df = pd.read_csv('/content/drive/MyDrive/test_df_numtadb.csv')



In [8]:
print(train_df['label'].unique())  # Check unique labels in the dataset
print(train_df['label'].nunique())  # Check the number of unique classes


[8 9 5 7 6 0 3 1 2 4]
10


In [9]:
print(f"Total training images: {len(train_df)}")
print(f"Total testing images: {len(test_df)}")
print(test_df['label'].nunique())

Total training images: 57636
Total testing images: 14409
10


In [10]:
import cv2
import numpy as np
import cv2.ximgproc as xip

def preprocess_image(file_path):
    img = cv2.imread(file_path, cv2.IMREAD_GRAYSCALE)
    if img is None:
        return np.zeros((28, 28))  # Placeholder for missing images

    # Apply Otsu's Thresholding
    _, img = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    # Resize image to 28x28
    img = cv2.resize(img, (28, 28), interpolation=cv2.INTER_AREA) / 255.0

    return img


In [11]:
# ======================== Load & Preprocess Images ========================
train_images = np.array([preprocess_image(fp) for fp in train_df['image_path'].values])


In [12]:
import cv2
import numpy as np
import multiprocessing
from joblib import Parallel, delayed
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import random



def augment_image(img):
    rows, cols = img.shape

    # Random Rotation
    angle = random.uniform(-15, 15)
    M = cv2.getRotationMatrix2D((cols / 2, rows / 2), angle, 1)
    rotated = cv2.warpAffine(img, M, (cols, rows))

    # Random Shifting
    tx = random.uniform(-2, 2)
    ty = random.uniform(-2, 2)
    M_shift = np.float32([[1, 0, tx], [0, 1, ty]])
    shifted = cv2.warpAffine(rotated, M_shift, (cols, rows))

    return shifted

def augment_dataset(train_images):
    """Applies augmentation to a dataset without parallel processing."""
    augmented_images = []
    for img in train_images:
        augmented = augment_image(img)
        augmented_images.append(augmented)
    return np.array(augmented_images)

# Apply augmentation to dataset
augmented_images = augment_dataset(train_images)




In [13]:
# cores = multiprocessing.cpu_count()
# print(cores)

In [14]:


!pip install mahotas


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 13.2 MB/s eta 0:00:00


In [16]:
from joblib import Parallel, delayed
import numpy as np
import multiprocessing
from skimage.feature import hog, local_binary_pattern
from skimage.filters import gabor
from mahotas.features import zernike_moments

# Feature Extraction
def extract_features_single(img):
    # Extract HOG features
    hog_features = hog(img, pixels_per_cell=(4, 4), cells_per_block=(2, 2), feature_vector=True)



    # Extract LBP features
    lbp = local_binary_pattern(img, P=8, R=1, method="uniform")
    lbp_hist, _ = np.histogram(lbp.ravel(), bins=np.arange(0, 10), range=(0, 10))
    lbp_hist = lbp_hist.astype("float")
    lbp_hist /= lbp_hist.sum()  # Normalize LBP histogram

    return np.hstack([hog_features, lbp_hist])

def extract_features(images):
    features = []
    for img in images:
        feat = extract_features_single(img)
        features.append(feat)
    return np.array(features)


# Apply feature extraction
augmented_features = extract_features(augmented_images)



In [17]:
# ======================== Data Preparation ========================
X_train_preprocessed = np.array([preprocess_image(fp) for fp in train_df['image_path']])
X_test_preprocessed = np.array([preprocess_image(fp) for fp in test_df['image_path']])

In [18]:
# Apply augmentation
X_train_augmented = augment_dataset(X_train_preprocessed)




In [19]:
# Extract features from augmented images
X_train_features = extract_features(X_train_augmented)
X_test_features = extract_features(X_test_preprocessed)  # No augmentation for test set

In [20]:
scaler = StandardScaler()
X_train_features = scaler.fit_transform(X_train_features)
X_test_features = scaler.transform(X_test_features)

encoder = LabelEncoder()
y_train = encoder.fit_transform(train_df['label'])
y_test = encoder.fit_transform(test_df['label'])

In [21]:
!pip install optuna



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.9/400.9 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.0/247.0 kB 6.6 MB/s eta 0:00:00


In [22]:
import cv2
import numpy as np
from glob import glob

def load_images(image_paths):
    data = []
    for path in image_paths:
        img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)  # Read as grayscale
        img = cv2.resize(img, (64, 64))  # Resize to a fixed size
        img = img.flatten()  # Convert to 1D array
        data.append(img)
    return np.array(data)

# Assuming X_train originally contains image file paths
X_train = load_images(X_train)


In [23]:
import optuna
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score

from sklearn.utils import resample
from optuna.samplers import TPESampler
from optuna.pruners import MedianPruner

# ======= Step 1: Sample a Subset of the Training Data =======
X_sampled, y_sampled = resample(
    X_train_features, y_train, n_samples=10000, stratify=y_train, random_state=42
)
# ======= Step 2: Define the Objective Function =======
def objective(trial):
    k = trial.suggest_int('n_neighbors', 3, 15)
    metric = trial.suggest_categorical('metric', ['euclidean', 'manhattan', 'minkowski'])
    weights = trial.suggest_categorical('weights', ['uniform', 'distance'])

    model = KNeighborsClassifier(n_neighbors=k, metric=metric, weights=weights)

    # Use cv=3 for faster tuning
    score = cross_val_score(model, X_sampled, y_sampled, cv=3, scoring='accuracy').mean()
    return score

In [24]:
# ======= Step 3: Create Study with Sampler and Pruner =======
study_knn = optuna.create_study(
    direction='maximize',
    sampler=TPESampler(seed=42),
    pruner=MedianPruner(n_startup_trials=10, n_warmup_steps=5)
)

# ======= Step 4: Optimize with Parallel Processing =======
study_knn.optimize(objective, n_trials=50, n_jobs=2)  # Use 2 cores (Colab-safe)

# ======= Step 5: Retrieve Best Parameters =======
best_knn_params = study_knn.best_params
print("Best KNN hyperparameters:", best_knn_params)






[I 2025-08-25 08:30:52,274] A new study created in memory with name: no-name-eea3854c-78eb-49fb-82d1-7fd3345f64ac
[I 2025-08-25 08:31:14,097] Trial 0 finished with value: 0.6728018932667114 and parameters: {'n_neighbors': 3, 'metric': 'euclidean', 'weights': 'distance'}. Best is trial 0 with value: 0.6728018932667114.
[I 2025-08-25 08:31:15,915] Trial 1 finished with value: 0.6563006030657056 and parameters: {'n_neighbors': 14, 'metric': 'euclidean', 'weights': 'uniform'}. Best is trial 0 with value: 0.6728018932667114.
[I 2025-08-25 08:31:34,839] Trial 2 finished with value: 0.6565005630737039 and parameters: {'n_neighbors': 11, 'metric': 'euclidean', 'weights': 'uniform'}. Best is trial 0 with value: 0.6728018932667114.
[I 2025-08-25 08:31:36,073] Trial 3 finished with value: 0.6380011426457582 and parameters: {'n_neighbors': 5, 'metric': 'minkowski', 'weights': 'uniform'}. Best is trial 0 with value: 0.6728018932667114.
[I 2025-08-25 08:31:45,596] Trial 5 finished with value: 0.6934

Best KNN hyperparameters: {'n_neighbors': 13, 'metric': 'euclidean', 'weights': 'distance'}


In [25]:
# Extract best values
best_k = best_knn_params['n_neighbors']
best_metric = best_knn_params['metric']
best_weights = best_knn_params['weights']

def knn(X_train, y_train, X_test, k=best_k, metric=best_metric, weights=best_weights):
    """Optimized KNN model."""

    # Convert labels to NumPy array (avoid indexing issues)
    y_train = np.array(y_train)

    # Fit KNN classifier with optimized parameters
    knn_model = KNeighborsClassifier(n_neighbors=k, metric=metric, weights=weights)
    knn_model.fit(X_train, y_train)

    # Predict on test set
    predictions = knn_model.predict(X_test)

    return predictions


In [26]:


# Run KNN with optimized parameters
knn_preds = knn(X_train_features, y_train, X_test_features)


In [27]:
X_train_combined = np.hstack((X_train_features, y_train.reshape(-1, 1)))
X_test_combined = np.hstack((X_test_features, knn_preds.reshape(-1, 1)))

In [28]:
import optuna
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam

feature_dim = X_train_combined.shape[1]  # Assuming X_train is your feature matrix


def objective(trial):
    """Objective function for optimizing neural network hyperparameters."""

    # Hyperparameters to optimize
    num_units_1 = trial.suggest_int('num_units_1', 128, 512, step=64)  # First Dense layer
    num_units_2 = trial.suggest_int('num_units_2', 64, 256, step=64)   # Second Dense layer
    num_units_3 = trial.suggest_int('num_units_3', 32, 128, step=32)   # Third Dense layer
    dropout_1 = trial.suggest_float('dropout_1', 0.2, 0.5)  # Dropout after first layer
    dropout_2 = trial.suggest_float('dropout_2', 0.2, 0.5)  # Dropout after second layer
    learning_rate = trial.suggest_loguniform('learning_rate', 0.0001, 0.001)  # Adam learning rate

    # Build model
    model = Sequential([
        Dense(num_units_1, activation='selu', input_shape=(feature_dim,)),
        BatchNormalization(),
        Dropout(dropout_1),
        Dense(num_units_2, activation='selu'),
        BatchNormalization(),
        Dropout(dropout_2),
        Dense(num_units_3, activation='selu'),
        BatchNormalization(),
        Dense(10, activation='softmax')
    ])

    # Compile model with tuned learning rate
    model.compile(optimizer=Adam(learning_rate=learning_rate),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    # Train model (Use fewer epochs for tuning)
    history = model.fit(
        X_train_combined, y_train,
        epochs=10,  # Keep epochs low for tuning
        batch_size=64,
        validation_split=0.2,
        verbose=0
    )

    # Return best validation accuracy
    return max(history.history['val_accuracy'])


In [29]:
study = optuna.create_study(direction='maximize')  # Maximize accuracy
study.optimize(objective, n_trials=20)  # Run 20 trials

# Get the best parameters
best_params = study.best_params
print("Best hyperparameters:", best_params)

[I 2025-08-25 08:48:25,105] A new study created in memory with name: no-name-df561f63-a311-4cb6-bb3f-c0e2a5762028
/tmp/ipython-input-3243621785.py:18: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 0.0001, 0.001)  # Adam learning rate
/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
[I 2025-08-25 08:51:08,191] Trial 0 finished with value: 0.9858605265617371 and parameters: {'num_units_1': 448, 'num_units_2': 128, 'num_units_3': 96, 'dropout_1': 0.3288322399958822, 'dropout_2': 0.3009081470817634

Best hyperparameters: {'num_units_1': 320, 'num_units_2': 192, 'num_units_3': 64, 'dropout_1': 0.44875006368671555, 'dropout_2': 0.2471380987724786, 'learning_rate': 0.000995942290051766}


In [30]:
# Unpack Optuna best parameters
num_units_1 = best_params['num_units_1']
num_units_2 = best_params['num_units_2']
num_units_3 = best_params['num_units_3']
dropout_1 = best_params['dropout_1']
dropout_2 = best_params['dropout_2']
learning_rate = best_params['learning_rate']
test_df['label'] = test_df['label'].astype(str)


efficientnet_preds = np.load("/content/drive/My Drive/test_predictions_numtadb.npy")



cv_scores = []
all_fold_preds = []

kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

for fold, (train_idx, val_idx) in enumerate(kfold.split(X_train_combined, y_train)):
    print(f"\n===== Training Fold {fold+1} =====")

    # Split into training and validation fold
    X_train_fold, X_val_fold = X_train_combined[train_idx], X_train_combined[val_idx]
    y_train_fold, y_val_fold = y_train[train_idx], y_train[val_idx]

    # Define MLP model
    model = Sequential([
        Dense(num_units_1, activation='selu', input_shape=(X_train_combined.shape[1],)),
        BatchNormalization(), Dropout(dropout_1),

        Dense(num_units_2, activation='selu'),
        BatchNormalization(), Dropout(dropout_2),

        Dense(num_units_3, activation='selu'),
        BatchNormalization(),

        Dense(10, activation='softmax')
    ])

    model.compile(optimizer=Adam(learning_rate=learning_rate),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    # Train on current fold
    model.fit(X_train_fold, y_train_fold, epochs=20, batch_size=64, verbose=0)

    # Predict on test set for current fold
    preds_proba = model.predict(X_test_combined)
    all_fold_preds.append(preds_proba)

# Average predicted probabilities across all folds
avg_preds_proba = np.mean(np.array(all_fold_preds), axis=0)

# Final predicted class labels from ensemble
fusionnet_preds = np.argmax(avg_preds_proba, axis=1)




test_df['label'] = test_df['label'].astype(str)

from statsmodels.stats.contingency_tables import mcnemar
import numpy as np

# Ensure true labels are encoded consistently
true_labels = encoder.transform(test_df['label'])  # Fixed line

# Check alignment
assert len(fusionnet_preds) == len(efficientnet_preds) == len(true_labels)

# Determine correctness
fusionnet_correct = fusionnet_preds == true_labels
efficientnet_correct = efficientnet_preds == true_labels

# Count mismatches
B = np.sum((fusionnet_correct == True) & (efficientnet_correct == False))
C = np.sum((fusionnet_correct == False) & (efficientnet_correct == True))

# Contingency table
table = [[0, B],
         [C, 0]]

# Run McNemar’s test
result = mcnemar(table, exact=False, correction=True)







===== Training Fold 1 =====


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


451/451 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step

===== Training Fold 2 =====
451/451 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step

===== Training Fold 3 =====
451/451 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step

===== Training Fold 4 =====
451/451 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step

===== Training Fold 5 =====
451/451 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step


In [32]:
# Print result
print("\n========= McNemar’s Test Result =========")
print(f"Contingency Table: [[0, {B}], [{C}, 0]]")
print(f"McNemar's Test Statistic: {result.statistic:.4f}")
print(f"P-value:  {result.pvalue:.4f}")




========= McNemar’s Test Result =========
Contingency Table: [[0, 179], [218, 0]]
McNemar's Test Statistic: 3.6372
P-value:  0.0503
